<a href="https://colab.research.google.com/github/Vakhranev/Pushkina/blob/main/%D0%9A%D0%B0%D0%B7%D0%B0%D0%BD%D1%86%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import re
import pandas as pd

# Словарь предметов с вариантами названий
SUBJECTS = {
    "Математика": ["Математика", "Алгебра", "Геометрия"],
    "Русский язык": ["Русский язык"],
    "Литература": ["Литература"],
    "История": ["История"],
    "Обществознание": ["Обществознание"],
    "География": ["География"],
    "Биология": ["Биология", "Ботаника"],
    "Химия": ["Химия"],
    "Физика": ["Физика"],
    "Информатика": ["Информатика", "Кибербезопасность"],
    "Иностранный язык": ["Английский язык", "Немецкий язык", "Французский язык", "Испанский язык"],
    "Музыка": ["Музыка"],
    "Изобразительное искусство": ["Изобразительное искусство", "ИЗО"],
    "Технология": ["Технология"],
    "ОБЖ": ["Основы безопасности жизнедеятельности", "ОБЖ"],
    "Физкультура": ["Физическая культура", "Физкультура"],
    "ОДНКР": ["ОДНКР", "ОДНКНР"],
    "Естествознание": ["Естествознание", "Окружающий мир", "Экология"],
}

BASE_URL = "https://rulex.kpfu.ru/booklist"
HEADERS = {"User-Agent": "Mozilla/5.0"}

# Функция для получения списка ссылок на "Список терминов"
def get_term_links():
    response = requests.get(BASE_URL, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    links = [
        "https://rulex.kpfu.ru" + a["href"]
        for a in soup.find_all("a", string="Список терминов")
    ]
    return links

# Функция для извлечения предмета из названия учебника
def extract_subject(title):
    for subject, variations in SUBJECTS.items():
        if any(variant in title for variant in variations):
            return subject
    return title  # Если предмет не найден, возвращаем полный заголовок

# Функция для извлечения минимального класса из заголовка
def extract_min_class(title):
    matches = re.findall(r"(\d+)\s*[-.]?\s*(?:й|го|гo)?\s*(?:кл[.]?|класс)", title, re.IGNORECASE)
    roman_matches = re.findall(r"\b([IVXLCDM]+)\b\s*[-.]?\s*(?:кл[.]?|класс)", title, re.IGNORECASE)
    classes = [int(match) for match in matches]
    roman_to_int = {"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5, "VI": 6, "VII": 7, "VIII": 8, "IX": 9, "X": 10}
    for match in roman_matches:
        if match in roman_to_int:
            classes.append(roman_to_int[match])
    return min(classes) if classes else None

# Функция для парсинга одной страницы
def parse_page(url):
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    class_info = soup.find("h4").text.strip() if soup.find("h4") else ""
    subject = extract_subject(class_info)
    min_class = extract_min_class(class_info)
    table = soup.find("table", class_="w3-table-all")
    terms = []
    if table:
        for row in table.find_all("tr")[1:]:
            cols = row.find_all("td")
            if len(cols) >= 2:
                term = cols[0].text.strip()
                frequency = int(cols[1].text.strip()) if cols[1].text.strip().isdigit() else 0
                terms.append((term, subject, min_class, frequency, class_info))  # Добавили class_info
    return terms, class_info

# Основная функция
def main():
    term_links = get_term_links()
    all_terms = []
    problematic_books = set()
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        results = executor.map(parse_page, term_links)
    for result, book_title in results:
        all_terms.extend(result)
        if any(term[2] is None for term in result):
            problematic_books.add(book_title)
    print("Учебники, у которых не удалось определить минимальный класс:")
    for book in problematic_books:
        print(book)
    term_dict = {}
    for term, subject, min_class, frequency, book in all_terms:
        if term not in term_dict:
            term_dict[term] = {"subjects": {}, "frequency": 0}
        if subject not in term_dict[term]["subjects"]:
            term_dict[term]["subjects"][subject] = min_class
        else:
            term_dict[term]["subjects"][subject] = min(term_dict[term]["subjects"][subject], min_class)
        term_dict[term]["frequency"] += frequency
    df = pd.DataFrame([
        (
            term,
            ", ".join(sorted(term_data["subjects"].keys())),
            ", ".join(str(term_data["subjects"][subject]) for subject in sorted(term_data["subjects"])),
            term_data["frequency"]
        )
        for term, term_data in term_dict.items()
    ], columns=["Слово или сочетание", "Предмет", "Минимальный класс, где встречается", "Частота"])
    df.to_excel("terms.xlsx", index=False)
    print("Файл terms.xlsx успешно создан!")
    search_term = "велит"

    # Ищем учебники, в которых встречается нужное слово
    books_with_term = set()
    for term, subject, min_class, frequency, book in all_terms:
        if term == search_term:
           books_with_term.add(book)

    # Выводим результат
    print(f"Слово '{search_term}' встречается в следующих учебниках:")
    for book in books_with_term:
        print(book)

if __name__ == "__main__":
    main()

Учебники, у которых не удалось определить минимальный класс:
Файл terms.xlsx успешно создан!
Слово 'велит' встречается в следующих учебниках:
История: Юдовская А. Я. Всеобщая история. История Нового времени, 1500-1800. 7 класс: учеб . для общеобразоват. организаций / А. Я. Юдовская, П. А. Баранов, Л. М. Ванюшкина ; под ред. А. А. Искендерова. - 6-е изд. - М. : Просвещение, 2018. - 319 с., [16] л. : ил., карт. - ISBN 978-5-09-055148-9.
История: Черникова, Татьяна Васильевна. История России с древнейших времён до начала XVI века. 6 класс: учебник/ Т. В. Черникова, К. П. Чиликин ; под общ. ред. В. Р. Мединского. – Москва : Просвещение, 2021. - 272 с. : ил., карты. ISBN 978-5-09-081267-2.
История: "Никишин В. О. История. Всеобщая история. История Древнего мира: учебник для 5 класса общеобразовательных организаций / В. О. Никишин, А. В. Стрелков, О. В. Томашевич, Ф.А. Михайловский; под науч. ред. С. П. Карпова. — М.: ООО «Русское слово — учебник», 2023. — 328 с.: ил. — (ФГОС 2021. Инновацио

In [4]:
!pip install spacy
!python -m spacy download ru_core_news_sm
import spacy

nlp = spacy.load("ru_core_news_sm")

def singularize(word):
    doc = nlp(word)
    return doc[0].lemma_

print(singularize("деревья"))  # Выведет "дерево"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 31.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
дерево


In [2]:
!pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 87.0 MB/s eta 0:00:00


In [11]:
import pandas as pd
import spacy
import pymorphy3

# Инициализация моделей
morph = pymorphy3.MorphAnalyzer()
nlp = spacy.load("ru_core_news_sm")

# Функция для получения формы в именительном падеже
def get_nominative_form(lemma):
    parses = morph.parse(lemma)
    for p in parses:
        if "nomn" in p.tag:
            return p
    return parses[0]

# Функция для преобразования фразы в единственное число
def to_singular(phrase):
    doc = nlp(phrase)
    words = [token.text_with_ws for token in doc]

    # Проверяем, содержит ли словосочетание существительное в именительном падеже и единственном числе
    for token in doc:
        if token.pos_ == "NOUN" and "Nom" in token.morph.get("Case") and "Sing" in token.morph.get("Number"):
            return None

    # Проверяем наличие двух существительных и одного прилагательного
    nouns = [token for token in doc if token.pos_ == "NOUN"]
    adjectives = [token for token in doc if token.pos_ == "ADJ"]

    if len(nouns) == 2 and len(adjectives) == 1:  # Проверяем структуру "существительное + прилагательное + существительное"
        first_noun = nouns[0]
        adj = adjectives[0]
        second_noun = nouns[1]

        if (
            "Nom" in first_noun.morph.get("Case")  # Первое существительное в именительном падеже
            and "Nom" not in second_noun.morph.get("Case")  # Второе существительное в косвенном падеже
            and first_noun.i < adj.i < second_noun.i  # Порядок: существительное, прилагательное, существительное
        ):
            # Если структура соответствует условию, проверяем число первого существительного
            if "Plur" in first_noun.morph.get("Number"):  # Если первое существительное во множественном числе
                singular_form = get_nominative_form(first_noun.text).inflect({"sing"})
                if singular_form:
                    words[first_noun.i] = singular_form.word + first_noun.whitespace_  # Изменяем первое существительное
            return "".join(words).strip()  # Возвращаем без изменений прилагательное и второе существительное

    singular_noun = None

    # Обрабатываем существительное во множественном числе
    for i, token in enumerate(doc):
        if token.pos_ == "NOUN" and "Plur" in token.morph.get("Number"):
            lemma = token.lemma_
            if lemma in {"анда", "чомпи"}:
                return None
            else:
                parses = morph.parse(lemma)
                for p in parses:
                    if "Pltm" in p.tag:
                        return None

            singular_form = get_nominative_form(lemma).inflect({"sing"})
            if singular_form:
                singular_noun = singular_form.word
                words[i] = singular_noun + token.whitespace_

                # Приводим прилагательные
                for j, adj_token in enumerate(doc):
                    if adj_token.pos_ == "ADJ" and (adj_token.head == token or adj_token.i > token.i):
                        adj_parse = morph.parse(adj_token.text)[0]
                        gender = singular_form.tag.gender
                        case = singular_form.tag.case
                        adj_form = adj_parse.inflect({"sing", gender, case})
                        if adj_form:
                            words[j] = adj_form.word + adj_token.whitespace_

    singular_phrase = "".join(words).strip()
    if singular_phrase == phrase:
        return None
    return singular_phrase

# Чтение таблицы
file_path = "/content/terms (5).xlsx"
df = pd.read_excel(file_path)

# Применяем к столбцу
df["Единственное число"] = df["Слово или сочетание"].astype(str).apply(to_singular)

# Сохраняем результат
output_path = "/content/terms_singular.xlsx"
df.to_excel(output_path, index=False)

print(f"Обработанный файл сохранён как {output_path}")

Обработанный файл сохранён как /content/terms_singular.xlsx


In [10]:
import spacy
import pymorphy3

# Инициализация моделей
morph = pymorphy3.MorphAnalyzer()
nlp = spacy.load("ru_core_news_sm")

# Функция для получения формы слова в именительном падеже
def get_nominative_form(word):
    parses = morph.parse(word)
    for parse in parses:
        if "nomn" in parse.tag:  # Ищем именительный падеж
            return parse
    return parses[0]  # Если именительный падеж не найден, возвращаем первый разбор

# Функция для изменения числа существительного
def to_singular_nominative(word):
    parses = morph.parse(word)
    for parse in parses:
        if "nomn" in parse.tag and "plur" in parse.tag:  # Именительный падеж и множественное число
            singular_form = parse.inflect({"sing"})
            if singular_form:
                return singular_form.word
    return word

# Основная функция
def process_phrase(phrase):
    doc = nlp(phrase)
    words = [token.text_with_ws for token in doc]  # Сохраняем пробелы между словами
    nouns = [token for token in doc if token.pos_ == "NOUN"]
    adjectives = [token for token in doc if token.pos_ == "ADJ"]

    if len(nouns) == 2 and len(adjectives) == 1:  # Проверяем структуру "существительное + прилагательное + существительное"
        first_noun = nouns[0]
        adj = adjectives[0]
        second_noun = nouns[1]

        if (
            "Nom" in first_noun.morph.get("Case")  # Первое существительное в именительном падеже
            and "Nom" not in second_noun.morph.get("Case")  # Второе существительное в косвенном падеже
            and first_noun.i < adj.i < second_noun.i  # Порядок: существительное, прилагательное, существительное
        ):
            # Если структура соответствует условию, проверяем число первого существительного
            if "Plur" in first_noun.morph.get("Number"):  # Если первое существительное во множественном числе
                singular_form = to_singular_nominative(first_noun.text)
                if singular_form:
                    words[first_noun.i] = singular_form + first_noun.whitespace_  # Изменяем первое существительное
            return "".join(words).strip()  # Возвращаем без изменений прилагательное и второе существительное

    # Преобразование для остальных случаев
    for token in doc:
        if token.pos_ == "NOUN" and "Nom" in token.morph.get("Case"):  # Существительное в именительном падеже
            if "Plur" in token.morph.get("Number"):  # Если существительное во множественном числе
                singular_form = to_singular_nominative(token.text)
                if singular_form:
                    words[token.i] = singular_form + token.whitespace_

                    # Приводим согласованные прилагательные к единственному числу
                    for adj_token in doc:
                        if adj_token.pos_ == "ADJ" and adj_token.head == token:  # Прилагательное согласовано с существительным
                            adj_parse = morph.parse(adj_token.text)[0]
                            singular_parse = morph.parse(singular_form)[0]
                            gender = singular_parse.tag.gender  # Род существительного
                            case = singular_parse.tag.case  # Падеж существительного
                            adj_form = adj_parse.inflect({"sing", gender, case})  # Приводим к числу, роду и падежу
                            if adj_form:
                                words[adj_token.i] = adj_form.word + adj_token.whitespace_

    return "".join(words).strip()

# Пример использования
phrases = [
    "органы внутренних дел",  # Прилагательное должно измениться
    "действия, арифметические",  # Без изменений, т.к. структура "сущ + прил + сущ"
    "арифметические действия",  # Преобразуется
    "музыкальные инструменты",  # Преобразуется
]

for phrase in phrases:
    processed_phrase = process_phrase(phrase)
    print(f"Оригинальная фраза: {phrase}")
    print(f"Преобразованная фраза: {processed_phrase}")
    print()

Оригинальная фраза: органы внутренних дел
Преобразованная фраза: орган внутренних дел

Оригинальная фраза: действия, арифметические
Преобразованная фраза: действие, арифметические

Оригинальная фраза: арифметические действия
Преобразованная фраза: арифметическое действие

Оригинальная фраза: музыкальные инструменты
Преобразованная фраза: музыкального инструмент

